## Контентная фильтрация
1. Получаем описания фильмов и другие метаданные оф ильмах (год съёмки, название)
2. Объединяем описание фильма, название фильмы и другие метаданные в одну строку (строка описания)
3. Разбиваем строку на слова
4. Удаляем незначимые слова (предлоги например)
5. Выполняем морфологическую обработку (чтобы слова супергеройский и супергерой считались одинаковыми, здесь это не реализовано)
6. С помощью алгоритма LDA получаем схожесть между строками описания (а также между фильмами)
7. Получаем топ фильмов по оценкам данного пользователя
8. Ищем фильмы которые имеют высокое соответствие с оценёнными фильмами пользователя
9. Предсказываем оценку этим фильмам изходя из оценки пользователя текущего фильма (фильмы для которого ищутся похожие фильмы), средней оценки пользователя и схожести фильмов.
10. Возвращаем топ по предсказанной оценке.

In [77]:
import requests
import time
import pandas as pd
import numpy as np
from decimal import Decimal

In [2]:
# class MovieDescriptions(models.Model):
#     movie_id = models.CharField(max_length=16)
#     imdb_id = models.CharField(max_length=16)
#     title = models.CharField(max_length=512)
#     description = models.CharField(max_length=1024)
#     genres = models.CharField(max_length=512, default='')
#     lda_vector = models.CharField(max_length=56, null=True)
#     sim_list = models.CharField(max_length=512, default='')

In [3]:
def get_descriptions(df):

    url = """https://api.themoviedb.org/3/discover/movie?primary_release_date.gte={}&api_key={}&page={}"""
    api_key = "e100e79e22d8dfaff6602ce5611fa2d3"
    NUMBER_OF_PAGES = 5
    start_date = "1970-01-01"

    #MovieDescriptions.objects.all().delete()

    for page in range(1, NUMBER_OF_PAGES):
        formated_url = url.format(start_date, api_key, page)
        print(formated_url)
        r = requests.get(formated_url)
        for film in r.json()['results']:
#             print(film)
#             print(df.index)
            df.loc[df.shape[0]] = [film['id'],film['title'],film['overview'],film['genre_ids'],film['release_date']]
#             id = film['id']
#             md = MovieDescriptions.objects.get_or_create(movie_id=id)[0]

#             md.imdb_id = get_imdb_id(id)
#             md.title = film['title']
#             md.description = film['overview']
#             md.genres = film['genre_ids']
#             if None != md.imdb_id:
#                 md.save()

        time.sleep(1)

#         print("{}: {}".format(page, r.json()))

In [4]:
# get_descriptions()

In [5]:
columns = ['movie_id', 'title', 'overview', 'genre_ids',  'release_date']
M = pd.DataFrame([],columns=columns)
get_descriptions(M)
print(M)

https://api.themoviedb.org/3/discover/movie?primary_release_date.gte=1970-01-01&api_key=e100e79e22d8dfaff6602ce5611fa2d3&page=1
https://api.themoviedb.org/3/discover/movie?primary_release_date.gte=1970-01-01&api_key=e100e79e22d8dfaff6602ce5611fa2d3&page=2
https://api.themoviedb.org/3/discover/movie?primary_release_date.gte=1970-01-01&api_key=e100e79e22d8dfaff6602ce5611fa2d3&page=3
https://api.themoviedb.org/3/discover/movie?primary_release_date.gte=1970-01-01&api_key=e100e79e22d8dfaff6602ce5611fa2d3&page=4
   movie_id                           title  \
0    505642  Black Panther: Wakanda Forever   
1    436270                      Black Adam   
2    663712                     Terrifier 2   
3    829280                  Enola Holmes 2   
4    675054                      MexZombies   
..      ...                             ...   
75   508947                     Turning Red   
76   579974                             RRR   
77   962232             Beyond the Universe   
78  1024530       

In [42]:
# import sys
# !{sys.executable} -m pip install stop_words
# !{sys.executable} -m pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.23
    Uninstalling Cython-0.29.23:
      Successfully uninstalled Cython-0.29.23


In [6]:
from stop_words import get_stop_words
import gensim
from nltk.tokenize import RegexpTokenizer
# tqdm показывает прогресс бар при выполнении итератора
from tqdm import tqdm

In [7]:
def load_data():
    columns = ['movie_id', 'title', 'overview', 'genre_ids',  'release_date']
    docs = pd.DataFrame([],columns=columns)
    get_descriptions(docs)
    
#     docs = list(MovieDescriptions.objects.all())
    data = ["{}, {}, {}".format(row['title'], row['genre_ids'], row['overview']) for index, row in docs.iterrows()]

    if len(data) == 0:
        print("No descriptions were found")
    return data, docs

In [8]:
#     Удаление не нужных слов (различные предлоги, вводные слова, общие слова)
def remove_stopwords(words):

#         так как текст на английском удаляем английский слова
        en_stop = get_stop_words('en')
# остаются только значимые слова (которые несут смысл)
        mean_words = [token for token in words if token not in en_stop]
        return mean_words

In [9]:
#     n_topics количество тем на которые разделяет документы LDA
def build_lda_model(data, docs, n_topics=5):

        texts = []
#         разбиваем текст на слова
        tokenizer = RegexpTokenizer(r'\w+')
#     для каждого фильма формируется одна строка метаданных d 
        for d in tqdm(data):
#         преобразование строки к нижнему регистру
            raw = d.lower()
# разбиение строки на слова
            words = tokenizer.tokenize(raw)

            mean_words = remove_stopwords(words)

            stemmed_words = mean_words
            
#             можно также удалить окончание у слов, чтобы слова супергерой и супергеройский считались одинаковыми
            #stemmer = PorterStemmer()
            #stemmed_words = [stemmer.stem(token) for token in stopped_words]

            texts.append(stemmed_words)

        print(len(texts))
        dictionary = gensim.corpora.Dictionary(texts)

        corpus = [dictionary.doc2bow(text) for text in texts]

        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary,
                                                 num_topics=n_topics)

        index = gensim.similarities.MatrixSimilarity(corpus)

#         self.save_lda_model(lda_model, corpus, dictionary, index)
#         self.save_similarities(index, docs)

        return dictionary, texts, lda_model,index,corpus

In [10]:
data, docs = load_data()

https://api.themoviedb.org/3/discover/movie?primary_release_date.gte=1970-01-01&api_key=e100e79e22d8dfaff6602ce5611fa2d3&page=1
https://api.themoviedb.org/3/discover/movie?primary_release_date.gte=1970-01-01&api_key=e100e79e22d8dfaff6602ce5611fa2d3&page=2
https://api.themoviedb.org/3/discover/movie?primary_release_date.gte=1970-01-01&api_key=e100e79e22d8dfaff6602ce5611fa2d3&page=3
https://api.themoviedb.org/3/discover/movie?primary_release_date.gte=1970-01-01&api_key=e100e79e22d8dfaff6602ce5611fa2d3&page=4


In [11]:
print(data)

['Black Panther: Wakanda Forever, [28, 12, 878], Queen Ramonda, Shuri, M’Baku, Okoye and the Dora Milaje fight to protect their nation from intervening world powers in the wake of King T’Challa’s death. As the Wakandans strive to embrace their next chapter, the heroes must band together with the help of War Dog Nakia and Everett Ross and forge a new path for the kingdom of Wakanda.', 'Black Adam, [28, 14, 878], Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods—and imprisoned just as quickly—Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.', "Terrifier 2, [27, 53], After being resurrected by a sinister entity, Art the Clown returns to Miles County where he must hunt down and destroy a teenage girl and her younger brother on Halloween night.  As the body count rises, the siblings fight to stay alive while uncovering the true nature of Art's evil intent.", 'Enola Holmes 2, [9648, 12, 80], Now a

In [12]:
dictionary, texts, lda_model,index,corpus = build_lda_model(data,docs)

100%|████████████████████████████████████████████████████████████████████████████████| 80/80 [00:00<00:00, 6171.27it/s]

80


In [13]:
print(len(data))
print(data[0])

80
Black Panther: Wakanda Forever, [28, 12, 878], Queen Ramonda, Shuri, M’Baku, Okoye and the Dora Milaje fight to protect their nation from intervening world powers in the wake of King T’Challa’s death. As the Wakandans strive to embrace their next chapter, the heroes must band together with the help of War Dog Nakia and Everett Ross and forge a new path for the kingdom of Wakanda.


In [14]:
# словарь содержит все слова из всех переданных строк описаний
# количество слов
print(len(dictionary))

1433


In [15]:
# corpus содержит список кортежей вида: (индекс_слова, количество_повторений_слова)
# Например мы видим кортеж: (825, 4). Это означает, что слово с индексом 825 встречается 4 раза во всех строках описаний
# Получаем это слово: dictionary[825], это слово "twin". Выполняем команду Ctrl+F в браузере и получаем подтверждение, что это слово встречается 4 раза.
print(corpus)
print(dictionary[825])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1)], [(1, 1), (2, 1), (5, 2), (31, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)], [(13, 1), (23, 1), (36, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1)], [(0, 1), (16, 1), (68, 1), (74, 1), (81, 1)

In [16]:
print(dir(lda_model))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_apply', '_load_specials', '_save_specials', '_smart_save', 'add_lifecycle_event', 'alpha', 'bound', 'callbacks', 'chunksize', 'clear', 'decay', 'diff', 'dispatcher', 'distributed', 'do_estep', 'do_mstep', 'dtype', 'eta', 'eval_every', 'expElogbeta', 'gamma_threshold', 'get_document_topics', 'get_term_topics', 'get_topic_terms', 'get_topics', 'id2word', 'inference', 'init_dir_prior', 'iterations', 'lifecycle_events', 'load', 'log_perplexity', 'minimum_phi_value', 'minimum_probability', 'num_terms', 'num_topics', 'num_updates', 'numworkers', 'offset', 'optimize_alpha', 'optimize_eta', 'passes', 'per_word_topics',

In [17]:
print(dir(lda_model.state))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_load_specials', '_save_specials', '_smart_save', 'add_lifecycle_event', 'blend', 'blend2', 'dtype', 'eta', 'get_Elogbeta', 'get_lambda', 'load', 'merge', 'numdocs', 'reset', 'save', 'sstats']


In [18]:
# извлечь данные из модели не удалось, но она должна хранить внутри себя список состоящий из кортежей, вида: (индекс_слова,tf-idf_этого_слова)
print(lda_model[corpus])

In [19]:
# index (принимает параметр query - новая строка-описание) используется для расчёта сооотвествия и возвращает список содержащий кортежи вида: (id_строки_описания, соответсвие_строке_описанию_query);
# Максимальное соотвествие 1 минимальное 0
# то есть с помощью index мы получаем похожие строки-описания
print(index)
from scipy.sparse import coo_matrix
# получаем разряженную матрицу???
coo = coo_matrix(index)
print(coo)
# сжимаем её ??? (удаляем нулевые значения)
csr = coo.tocsr()
print(csr)

MatrixSimilarity<80 docs, 1433 features>
  (0, 0)	0.99999976
  (0, 1)	0.14920942
  (0, 2)	0.070456706
  (0, 3)	0.04829453
  (0, 4)	0.11886433
  (0, 5)	0.06952347
  (0, 6)	0.05714286
  (0, 7)	0.123960264
  (0, 8)	0.08817335
  (0, 9)	0.041675713
  (0, 10)	0.025253814
  (0, 11)	0.06234797
  (0, 14)	0.046348978
  (0, 16)	0.020004002
  (0, 17)	0.08924215
  (0, 18)	0.107989855
  (0, 19)	0.111590505
  (0, 20)	0.04357102
  (0, 22)	0.08518355
  (0, 23)	0.08924215
  (0, 24)	0.09352196
  (0, 26)	0.112066366
  (0, 27)	0.041239306
  (0, 28)	0.07824608
  (0, 29)	0.053994928
  :	:
  (79, 50)	0.09166984
  (79, 51)	0.020619653
  (79, 52)	0.058321185
  (79, 53)	0.17857142
  (79, 54)	0.025717225
  (79, 55)	0.05143445
  (79, 56)	0.07319251
  (79, 57)	0.025253812
  (79, 58)	0.035399616
  (79, 59)	0.06419407
  (79, 60)	0.029880714
  (79, 61)	0.039405517
  (79, 62)	0.042796046
  (79, 63)	0.025253812
  (79, 64)	0.08547043
  (79, 67)	0.03149704
  (79, 68)	0.025717225
  (79, 69)	0.03509312
  (79, 70)	0.10714285

In [20]:
# print(docs.head())
print(docs.iloc[2]['movie_id'])

663712


In [21]:
from datetime import datetime
from scipy.sparse import coo_matrix
def create_movie_similarity_df(index,docs):
    # получаем разряженную матрицу???
    coo = coo_matrix(index)
    print(coo)
    # сжимаем её ??? (удаляем нулевые значения)
    csr = coo.tocsr()
    print(csr)
    print(f'{coo.count_nonzero()} similarities to save')
    no_saved = 0
    min_sim=0.1
    start_time = datetime.now()
    xs, ys = coo.nonzero()
    print(xs)
    
    
    columns = ['source_id', 'target_id', 'similarity']
    M = pd.DataFrame([],columns=columns)
    
    for x, y in zip(xs, ys):
    #     x это первый элемент кортежа, y второй; x,y это id строк описаний между которыми вычисляется сходство
    #     print(x,y)
    # нет смысл сравнить строку описания с самой собой это вернёт 1 или близко к этому
        if x == y:
            continue


        sim = float(csr[x, y])
    #     сходство строки описания x и y, такое же как и, y и x; что логично=)
        if (float(csr[x, y]) != float(csr[y, x])):
            print("КАК же это так")
    #     print(x)
    #     print(docs.loc[[x]]['movie_id'][0])
        x_id = str(docs.iloc[x]['movie_id'])
        y_id = str(docs.iloc[y]['movie_id'])
    #     Отбрасываем совсем непохожие элементы
        if sim < min_sim:
            continue

    # #     LdaSimilarity(created, x_id, y_id, sim).save()
        print('source: {} target: {} sim: {}'.format(x_id,y_id,sim))
        M.loc[M.shape[0]] = [x_id,y_id,sim]
        no_saved += 1

    print('{} Similarity items saved, done in {} seconds'.format(no_saved, datetime.now() - start_time))
    return M

In [27]:
similarity_table =create_movie_similarity_df(index,docs)

  (0, 0)	0.99999976
  (0, 1)	0.14920942
  (0, 2)	0.070456706
  (0, 3)	0.04829453
  (0, 4)	0.11886433
  (0, 5)	0.06952347
  (0, 6)	0.05714286
  (0, 7)	0.123960264
  (0, 8)	0.08817335
  (0, 9)	0.041675713
  (0, 10)	0.025253814
  (0, 11)	0.06234797
  (0, 14)	0.046348978
  (0, 16)	0.020004002
  (0, 17)	0.08924215
  (0, 18)	0.107989855
  (0, 19)	0.111590505
  (0, 20)	0.04357102
  (0, 22)	0.08518355
  (0, 23)	0.08924215
  (0, 24)	0.09352196
  (0, 26)	0.112066366
  (0, 27)	0.041239306
  (0, 28)	0.07824608
  (0, 29)	0.053994928
  :	:
  (79, 50)	0.09166984
  (79, 51)	0.020619653
  (79, 52)	0.058321185
  (79, 53)	0.17857142
  (79, 54)	0.025717225
  (79, 55)	0.05143445
  (79, 56)	0.07319251
  (79, 57)	0.025253812
  (79, 58)	0.035399616
  (79, 59)	0.06419407
  (79, 60)	0.029880714
  (79, 61)	0.039405517
  (79, 62)	0.042796046
  (79, 63)	0.025253812
  (79, 64)	0.08547043
  (79, 67)	0.03149704
  (79, 68)	0.025717225
  (79, 69)	0.03509312
  (79, 70)	0.10714285
  (79, 72)	0.05455447
  (79, 73)	0.08748

source: 616037 target: 505642 sim: 0.11159050464630127
source: 616037 target: 675054 sim: 0.10832372307777405
source: 616037 target: 642885 sim: 0.1626567393541336
source: 616037 target: 718930 sim: 0.11644449830055237
source: 616037 target: 916719 sim: 0.1002194732427597
source: 616037 target: 811061 sim: 0.11644449830055237
source: 361743 target: 718930 sim: 0.11366572976112366
source: 420634 target: 663712 sim: 0.3514997959136963
source: 420634 target: 830788 sim: 0.10101524740457535
source: 420634 target: 848058 sim: 0.1380131095647812
source: 420634 target: 760741 sim: 0.10482848435640335
source: 420634 target: 928773 sim: 0.2849014401435852
source: 420634 target: 928015 sim: 0.11338934302330017
source: 420634 target: 86328 sim: 0.4285714030265808
source: 420634 target: 916719 sim: 0.10286889970302582
source: 420634 target: 811061 sim: 0.11952285468578339
source: 420634 target: 414906 sim: 0.11396057158708572
source: 420634 target: 1024535 sim: 0.1259881556034088
source: 420634 ta

source: 791155 target: 585511 sim: 0.13201966881752014
source: 539681 target: 505642 sim: 0.1322600245475769
source: 539681 target: 436270 sim: 0.10744307190179825
source: 539681 target: 675054 sim: 0.17118419706821442
source: 539681 target: 634649 sim: 0.12354154884815216
source: 539681 target: 610150 sim: 0.17757120728492737
source: 948276 target: 663712 sim: 0.10762439668178558
source: 948276 target: 718930 sim: 0.13012000918388367
source: 948276 target: 848526 sim: 0.11664237082004547
source: 948276 target: 848058 sim: 0.11268723756074905
source: 948276 target: 852046 sim: 0.2182178944349289
source: 948276 target: 848791 sim: 0.13631963729858398
source: 948276 target: 916719 sim: 0.1259881556034088
source: 948276 target: 579974 sim: 0.11268723756074905
source: 86328 target: 663712 sim: 0.21968737244606018
source: 86328 target: 1033107 sim: 0.17928428947925568
source: 86328 target: 420634 sim: 0.4285714030265808
source: 86328 target: 879538 sim: 0.14638501405715942
source: 86328 tar

In [ ]:
# class Rating(models.Model):
#     user_id = models.CharField(max_length=16)
#     movie_id = models.CharField(max_length=16)
#     rating = models.DecimalField(decimal_places=2, max_digits=4)
#     rating_timestamp = models.DateTimeField()
#     type = models.CharField(max_length=8, default='explicit')

# Получаем топ элементов по оценкам данного пользователя
# active_user_items = Rating.objects.filter(user_id=user_id).order_by('-rating')[:100]

# content_sims = dict()
# словарь вида {movie_id:rating}
movie_ids = {movie['movie_id']: movie['rating'] for movie in active_user_items}
user_mean = sum(movie_ids.values()) / len(movie_ids)

# используя таблицу созданную в create_movie_similarity_df ищем элементы по source_id, исключаем эти же элементы из target_id и отбрасываем элементы с маленькой схожестью (хотя это не нужны, мы сделали это раньше)
sims = LdaSimilarity.objects.filter(Q(source__in=movie_ids.keys())& ~Q(target__in=movie_ids.keys())& Q(similarity__gt=self.min_sim))

# берём топ по схожести
sims = sims.order_by('-similarity')[:self.max_candidates]

recs = dict()
# s - [source_id,target_id,similarity]
targets = set(s.target for s in sims)
# target_id
for target in targets:
    pre = 0
    sim_sum = 0
#     получаем строки s с target
    rated_items = [i for i in sims if i.target == target]
    if len(rated_items) > 0:
        # sim_item - [source_id,target_id,similarity]
        for sim_item in rated_items:
#             вычитаем из оценки пользователя его среднюю оценку
            r = Decimal(movie_ids[sim_item.source] - user_mean)
#     умножаем сходство на оценку
            pre += sim_item.similarity * r
#     суммарное сходство
            sim_sum += sim_item.similarity
#     если есть сходство (TRUE всегда)
        if sim_sum > 0:
#         возврашаем предполагаемую оценку и схожие фильмы
            recs[target] = { 'prediction': Decimal(user_mean) + pre/sim_sum,'sim_items': [r.source for r in rated_items]}
#     сортируем по убыванию преполагаемой оценки
return sorted(recs.items(),key=lambda item: -float(item[1]['prediction']))[:num]

In [83]:
# Пример
columns = ['user_id', 'movie_id', 'rating']
user_ratings = pd.DataFrame([
    ['100','848791',5.7],
     ['100','579974',3.7],
     ['100','834734',6.7],
     ['100','86328',5.9],
     ['100','879538',7.7],
     ['100','616037',9.7],
     ['100','830788',8.7],
     ['100','966220',5.5],
     ['100','675054',5.2],
],columns=columns)

user_mean = user_ratings['rating'].mean()

user_ratings.sort_values(['rating'], ascending=[0],inplace=True)
user_ratings.reset_index(drop=True, inplace=True)
print(user_ratings)

in_source=similarity_table['source_id'].isin(user_ratings['movie_id']) 
not_in_target=~similarity_table['target_id'].isin(user_ratings['movie_id'])

top_by_sims = similarity_table.loc[in_source &  not_in_target].sort_values(['similarity'], ascending=[0])[:30]
top_by_sims.reset_index(drop=True, inplace=True)
print(top_by_sims)

columns = ['target_id', 'prediction', 'sim_movies']
recs = pd.DataFrame([],columns=columns)
targets = top_by_sims['target_id']

for target_row in targets:
    pre = 0
    sim_sum = 0
#     print(target_row)
    rated_items = top_by_sims[top_by_sims['target_id']==target_row]
#     print(rated_items)
    if len(rated_items) > 0:
        for index, sim_item in rated_items.iterrows():
            user_rating = user_ratings[user_ratings['movie_id']==sim_item['source_id']].iloc[0]['rating']
        
            r = user_rating- user_mean
            pre += sim_item.similarity * r
            sim_sum += sim_item.similarity
        if sim_sum > 0:
#               print(rated_items['source_id'].values)
            recs.loc[recs.shape[0]] = [target_row,user_mean + pre/sim_sum,rated_items['source_id'].values]
#             recs[target] = { 'prediction': Decimal(user_mean) + pre/sim_sum,'sim_items': [r.source for r in rated_items]}
# return sorted(recs.items(),key=lambda item: -float(item[1]['prediction']))[:num]
recs.sort_values(['prediction'], ascending=[0],inplace=True)
recs.reset_index(drop=True, inplace=True)
print(recs)

  user_id movie_id  rating
0     100   616037     9.7
1     100   830788     8.7
2     100   879538     7.7
3     100   834734     6.7
4     100    86328     5.9
5     100   848791     5.7
6     100   966220     5.5
7     100   675054     5.2
8     100   579974     3.7
   source_id target_id  similarity
0      86328    420634    0.428571
1      86328    663712    0.219687
2     830788    760741    0.185312
3     675054    960704    0.181568
4     834734    811061    0.179284
5      86328   1033107    0.179284
6      86328    924482    0.178571
7     675054    539681    0.171184
8      86328    928773    0.170941
9     830788    811061    0.169031
10    830788    760161    0.167038
11    830788    979924    0.164957
12    616037    642885    0.162657
13    830788    928773    0.161165
14    830788    663712    0.155342
15    830788    848526    0.151523
16    966220    811061    0.145521
17    879538    760741    0.143223
18    834734    420634    0.142857
19    675054   1024530    0.13